In [1]:
import os
import datetime
from statistics import mean, stdev
proj_dir = os.path.dirname(os.getcwd())
os.chdir(proj_dir)
from flow_analysis import Participant_Flow

In [2]:
par_num = 1
par = Participant_Flow(par_num)

# Kernel Flow PC - Cognitive Task PC

print(f"Experiment order: {par.par_behav.exp_order}\n")

exp_name = "resting_state"
exp = par.par_behav.get_exp(exp_name)
exp_time_origin_ts = exp.start_ts
exp_time_origin_dt = datetime.datetime.fromtimestamp(exp_time_origin_ts)  # behav data filename and exp start marker timestamp
print(f"Experiment time origin: {exp_time_origin_dt}")
start_marker_sent_time = float(exp.marker_data["start_marker"]["sent_time"])
print(f"Start marker sent time: {start_marker_sent_time}")
start_marker_sent_time_abs = exp_time_origin_dt + datetime.timedelta(seconds=start_marker_sent_time)
print(f"Start marker sent time (absolute): {start_marker_sent_time_abs}")

# The behavioral start marker sent time (absolute) and 
# kernel start marker receive time (absolute) should be identical.

flow_time_origin = par.flow_session_dict["session_1001"].get_time_origin(offset=False)
print(f"Flow time origin: {flow_time_origin}\n")
session = par.par_behav.get_key_from_value(par.par_behav.session_dict, exp_name)
marker_df = par.flow_session_dict[session].get_marker_df()
print(f"Kernel marker data (original):\n{marker_df}\n")
marker_df_abs = par.create_abs_marker_df(session)
print(f"Kernel marker data (absolute):\n{marker_df_abs}")

row = marker_df_abs.loc[marker_df_abs["Marker"].str.startswith(exp_name)].reset_index()
kernel_start_marker_ts = row.loc[0, "Start timestamp"]
time_offset = kernel_start_marker_ts - (exp_time_origin_ts + start_marker_sent_time)
print(f"\nTime offset: {round(time_offset, 3)} seconds\n")

par_num = 1
par = Participant_Flow(par_num)
time_offset_list = []
print("Time difference:\n------------")
par.time_offset_dict

Experiment order: ['resting_state', 'go_no_go', 'video_narrative_sherlock', 'king_devick', 'vSAT', 'audio_narrative', 'n_back', 'tower_of_london', 'video_narrative_cmiyc']

Experiment time origin: 2022-06-15 14:43:17.132986
Start marker sent time: 11.6911
Start marker sent time (absolute): 2022-06-15 14:43:28.824086
Flow time origin: 2022-06-15 18:43:22

Kernel marker data (original):
     Timestamp    Duration  Value  Experiment
0    19.105127  441.473551    1.0        51.0
1  1120.861606  514.570869    1.0        81.0

Kernel marker data (absolute):
                           Marker  Start timestamp    Duration  End timestamp
0             resting_state_start     1.655319e+09  441.473551   1.655319e+09
1  video_narrative_sherlock_start     1.655320e+09  514.570869   1.655320e+09

Time offset: 12.281 seconds

Time difference:
------------


{'resting_state': 12.281041622161865,
 'go_no_go': 12.282827615737915,
 'video_narrative_sherlock': 12.284613609313965,
 'king_devick': 11.980223417282104,
 'vSAT': 11.980556726455688,
 'audio_narrative': 11.982890844345093,
 'n_back': 12.57655382156372,
 'tower_of_london': 12.580034732818604,
 'video_narrative_cmiyc': 12.58246636390686,
 'session_1001': 12.282827615737915,
 'session_1002': 11.981223662694296,
 'session_1003': 12.579684972763062}

In [5]:
num_pars = 15
time_diff_list = []
for i in range(num_pars):
    par_num = 1 + 1
    par = Participant_Flow(par_num + 1)
    for session, exp_list in par.par_behav.session_dict.items():
        for exp_name in exp_list:
            if exp_name.startswith("session_"):
                continue
            elif exp_name == "go_no_go":
                continue
            session_number = int(session.split("_")[1])
            session_time = par.time_offset_dict["session_" + str(session_number)]
            experiment_time = par.time_offset_dict[exp_name]
            time_diff = abs(experiment_time - session_time)
            time_diff_list.append(time_diff)

print(f"Min: {round(min(time_diff_list) * 1e3, 2)} milliseconds")
print(f"Max: {round(max(time_diff_list) * 1e3, 2)} milliseconds")
print(f"Mean: {round(mean(time_diff_list) * 1e3, 2)} ± {round(stdev(time_diff_list) * 1e3, 2)} milliseconds")

Min: 0.15 milliseconds
Max: 1.85 milliseconds
Mean: 0.72 ± 0.57 milliseconds
